# Writing Multifile Programs

Contents:
* Header Files
  * Using headers to break a single file into multiple files
* Build systems
  * CMake and Make
* Tools for writing larger programs:
  * References
  * Pointers
  * Maps
  * Classes and OOP in C++

## Header Files
* Header `.h` files hold related functions, methods, and class declarations.
* The corresponding definitions are placed in `.cpp` files.

The compiler considers a header declaration as a "promise" that the definition will be found later in the code. This means that when the compiler reaches a function athat hasn't been defined yet, it can continue on compiling until the definition is found.

This allows functions to be defined & declared in arbitrary order.

In order to obtain being able to use functions in arbitrary order, they would have to be declared at the top of a file. To avoid a single file from becomming cluttered with declarations and definitions for every function, it is customary to declare the functions in another file, called the header file. 

In [ ]:
// Header file: header+example.h
#ifndef HEADER_EXAMPLE_H
#define HEADER_EXAMPLE_H

void OuterFunction(int);
void InnerFunction(int);

#endif

In [ ]:
// Include the contents of header_example.h using quotes:
#include "header_example.h"

#include <iostream>
using std::cout;

void OuterFunction(int i) {
    InnerFunction(i);
}

void InnerFunction(int i) {
    cout << "The value of the integer is: " << i << "\n";
}

int a = 5;
OuterFunction(a);

* The #include statement for the header used quotes " " around the file name, and not angle brackets <>. We have stored the header in the same directory as the .cpp file, and the quotes tell the preprocessor to look for the file in the same directory as the current file - not in the usual set of directories where libraries are typically stored.


This is a preprocessor directive:
```cpp
#ifndef HEADER_EXAMPLE_H
#define HEADER_EXAMPLE_H
```
This is called an "include guard". Since the header will be included into another file, and #include just pastes contents into a file, the include guard prevents the same file from being pasted multiple times into another file.

**Note:** There are other ways to do this. Another common way is to use an [#pragma once preprocessor](https://en.wikipedia.org/wiki/Pragma_once) directive, but we won't cover that in detail here.

## Using Headers with Mulitple Files

`vect_add_one.h` and `vect_add_one.cpp`

In [ ]:
#ifndef VECT_ADD_ONE_H
#define VECT_ADD_ONE_H

#include <vector>
using std::vector;

// AddOneToEach method declaration.
void AddOneToEach(vector<int> &v);

#endif

In [ ]:
#include "vect_add_one.h"

void AddOneToEach(vector<int> &v) 
{
    for (auto& i: v) {
        i++;
    }
}

`increment_and_sum.h` and `increment_and_sum.cpp`

In [ ]:
#ifndef INCREMENT_AND_SUM_H
#define INCREMENT_AND_SUM_H

#include <vector>
using std::vector;

// IncrementAndComputeVectorSum method declaration.
int IncrementAndComputeVectorSum(vector<int> v);

#endif

In [ ]:
#include "vect_add_one.h"

int IncrementAndComputeVectorSum(vector<int> v) {
    int total = 0;
    AddOneToEach(v);

    for (auto i: v) {
        total += i;
    }
    return total;
}

`main.cpp`

In [ ]:
#include <iostream>
#include <vector>
#include "increment_and_sum.h"
using std::vector;
using std::cout;

int main() 
{
    vector<int> v{1, 2, 3, 4};
    int total = IncrementAndComputeVectorSum(v);
    cout << "The total is: " << total << "\n";
}

**Notes:**
* Some libraries, like `<vector>` are included in multiple files.
  * Each file is compiled alone and must have all the declarations and libraries necessary to compile, so the necessary libraries must be included. This is another reason why include guards are important - if multiple headers were included in main, each with the same #include `<vector>` statement, you wouldn't want the vector header pasted multiple times into the code.
* The g++ compile command from the "Run Code" button is:
```bash
g++ -std=c++17 ./code/main.cpp ./code/increment_and_sum.cpp ./code/vect_add_one.cpp && ./a.out
```
  * When compiling, each of the relevant .cpp files must be included in the compile command. The -std=c++17 specifies that we are using the C++ 17 standard (which happens automatically in the terminal).

# Build Systems

## Bjarne on Build Systems
* Different organizations have big & elaborate build processes to get their software produced consistently, coherently, and repetetively. 
* Start with CMake, and then go to more specific build systems. They are somewhat different and you have to learn a few.

## CMake and Make
So far, we've seen how to split code into multiple `.h` and `.cpp` files, and how to build them together using `g++`.

[Build systems](https://en.wikipedia.org/wiki/List_of_build_automation_software) are useful for large C++ projects with many files. They help to manage all files during the build process and allows large projects to be compiled with a few commands. 

Build systems are able to build projects efficiently by only recompiling files that have been changed.

Questions:
* What happens when you run `g++`?
* How does one use object files to compile only a single file at a time?
  * This can be used to compile only files that have changed and need to be re-compiled, as opposed to recompiling everything.
* How does one use `cmake` and `make`?
  * `cmake` is a [build system](https://cmake.org/success/) which is popular in large C++ projects. It simplifies the process of building projects and recompiling only the changed files.
  
### Object Files
When a project is compiled with `g++`, several distinct tasks are performed:
1. The preprocessor runs and executes any statement beginning with a hash symbol, e.g. `#include`.
  * This ensures all code is in the correct location and ready to compile.
2. Each file in the source code is compiled into an object file `.o`.
  * Object files are platform-specific machine code that will be used to create an executable.
3. Object files are _linked together_ ot make a single executable (e.g. `a.out`).

It is possible to have `g++` perform each of the steps separately by using the `-c` flag. 

1. Produce the `main.o` file:
```bash
g++ -c main.cpp
```
2. Convert the object file into an executable:
```bash
g++ main.o
```

Performing it on multiple `.cpp` files:
```bash
g++ -c *.cpp && g++ *.o
./a.out
```

The `*` operator is a wildcard, any matching file is selected.

### CMake and and Make
* CMake
  * open-source, platform-independent build system
  * uses text documents (`CMakeLists.txt` files) to manage build environments, like [make](https://en.wikipedia.org/wiki/Make_(software))
  
`CMakeLists.txt`
* Simple text configuration files that tell CMake how to build your project.
* There can be multiple `CMakeLists.txt` files in a project, often one file per directory that indicates how the files in that directory should be built.
* The files can be used to:
  * specify locations of necessary packages
  * set build flags
  * set environment variables
  * specify build target names and locations
  * and more.
  
  
#### Example
Directory:
* `CMakeLists.txt`
* `src`
  * `file1.cpp`
  * `file1.h`
  * `file2.cpp`
  * `file2.h`
  * `main.cpp`
* `build` (after creating this directory and building the project, we have):
  * Various files like: `CmakeCache.txt`, `Makefile`, `cmake_install.cmake`
  * Directories: `CMakeFiles`, `my_project`

1. Edit the `CMakeLists.txt` file:
```cmake
# Specify the minimum versions of cmake & C++ that are required to biuld the project
cmake_minimum_required(VERSION 3.5.1)
set(CMAKE_CXX_STANDARD 17) #  require C++17
# Name of the project (required)
project(my_project_name)
# NOTE: CMake has the ability to automatically find source files in a directory - but for now let's do it manually as an exercise:
# Add an executable to the project. Specify the executable name, and give the locations of all the source files that are needed.
add_executable(my_executable_name src/main.cpp src/increment_and_sum.cpp src/vect_add_one.cpp)
```
2. Create the `build` directory, navigate into it and run:
  1. `cmake ..`
    * This command uses the `CMakeLists.txt` file to configure the project and create a `Makefile` in the `build` directory.
  2. `make`
    * `make` finds the `Makefile` and uses the instructions there to build the project.
3. Run the executable:
  * `./my_project_name`
  
Whenever a file is changed, only the `make` command has to be run from the build folder. In general, CMake only needs to be run once for a project, unless the build options are being changed (e.g. using different buil flags, or changing where files are stored).

Make keeps track of which files have changed and compiles only those that need to be compiled before building.

**Note:** In case that CMake has to re-run, it can be helpful to delete the build directory and start from scratch. Otherwise, some environment variables may not be reset correctly.